In [37]:
from bs4 import BeautifulSoup
import unittest
import re

path = '/home/kl/Рабочий стол/Python_Course/creating_web_in_python_3_course/week2/BS_parser/wiki/Artificial_intelligence'

Answers
            ('wiki/Stone_Age', [13, 10, 12, 40]),
            ('wiki/Brain', [19, 5, 25, 11]),
            ('wiki/Artificial_intelligence', [8, 19, 13, 198]),
            ('wiki/Python_(programming_language)', [2, 5, 17, 41]),
            ('wiki/Spectrogram', [1, 2, 4, 7]),)

In [33]:
def get_images_amount(body):
    imgs = body.find_all('img')
    fit_imgs = len(
        [x for x in imgs if x.get('width') and int(x.get('width')) >= 200]
    )
    return fit_imgs

def get_headers_amount(body):
    headers = body.find_all(re.compile('^h[1-6]$'))
    count = 0
    for header in headers:
        children = header.find_all(recursive=False)
        if children:
            children_content = [x.getText() for x in children if x.getText()]
            try:
                first_letter = children_content[0][0]
                if first_letter in 'ETC':
                    count += 1
            except IndexError:
                pass
        else:
            try:
                first_letter = header.getText()[0]
                if first_letter in 'ETC':
                    count += 1
            except IndexError:
                pass
    return count


def get_max_links_len(body):
    max_count = 0
    all_links = body.find_all('a')
    for link in all_links:
        current_count = 1
        siblings = link.find_next_siblings()
        for sibling in siblings:
            if sibling.name == 'a':
                current_count += 1
                max_count = max(current_count, max_count)
            else:
                current_count = 0
    return max_count


def get_lists_num(body):
    count = 0
    all_lists = body.find_all(['ul', 'ol'])
    for tag in all_lists:
        if not tag.find_parents(['ul', 'ol']):
            count += 1
    return count


def parse(path_to_file):    
    with open(path,'r',encoding='utf-8') as html:
        soup = BeautifulSoup(html, "lxml")    
    
    body = soup.find(id="bodyContent")
#     print(body)
    imgs = get_images_amount(body)
    headers = get_headers_amount(body)
    linkslen = get_max_links_len(body)
    lists = get_lists_num(body)
    
    return [imgs, headers, linkslen, lists]

In [38]:
parse(path)

[8, 19, 13, 198]

In [12]:
from bs4 import BeautifulSoup
from decimal import Decimal


def convert(amount, cur_from, cur_to, date, requests):
    result = requests.get("http://www.cbr.ru/scripts/XML_daily.asp", {"date_req": date})
    soup = BeautifulSoup(result.content, 'xml')
    rates = {i.CharCode.string: (
            Decimal(i.Value.string.replace(',', '.')),
            int(i.Nominal.string)
        ) for i in soup('Valute')
    }
    rates['RUR'] = (Decimal(1), 1)

    result = amount * rates[cur_from][0] * rates[cur_to][1] / rates[cur_from][1] / rates[cur_to][0]
    return result.quantize(Decimal('.0001'))

In [8]:
from decimal import Decimal
Decimal('10')/Decimal('5')

Decimal('2')

In [14]:
import requests
from decimal import Decimal
from currency import convert


correct = Decimal('3754.8057')
result = convert(Decimal("1000.1000"), 'RUR', 'JPY', "17/02/2005",requests)
if result == correct:
    print("Correct")
else:
    print("Incorrect: %s != %s" % (result, correct))

TypeError: get() missing 1 required positional argument: 'url'

In [ ]:
import requests
from bs4 import BeautifulSoup

resp = requests.get("http://www.cbr.ru/scripts/XML_daily.asp")
soup = BeautifulSoup(resp.content, "xml")
soup

In [4]:
soup.find('CharCode', text='EUR').find_next_sibling('Value').string

'80,0598'

In [43]:
from bs4 import BeautifulSoup
from decimal import Decimal
cur_from = 'EUR'
cur_to = 'USD'

response = requests.get('http://www.cbr.ru/scripts/XML_daily.asp?')  # Использовать переданный requests
soup = BeautifulSoup(resp.content, "xml")

from_ = soup.find('CharCode', text=cur_from).find_next_sibling('Value').string
from_nominal = soup.find('CharCode', text=cur_from).find_next_sibling('Nominal').string
    
to_ = soup.find('CharCode', text=cur_to).find_next_sibling('Value').string
to_nominal = soup.find('CharCode', text=cur_to).find_next_sibling('Nominal').string

print('From {}:{}, Nominal:{}'.format(cur_from,from_,from_nominal))
print('To {}:{}, Nominal:{}'.format(cur_to,to_,to_nominal))

a = float(from_nominal)/float(to_nominal)
for i in to_:
    print(i)
# print('1 {} = {} {}'.format(cur_from,a,cur_to))

From EUR:80,0598, Nominal:1
To USD:73,9719, Nominal:1
7
3
,
9
7
1
9


In [16]:
path = '/home/kl/Рабочий стол/Python_Course/creating_web_in_python_3_course/week2/BS_parser/wiki/Stone_Age'

In [23]:
from bs4 import BeautifulSoup
import unittest

def parse(path):
    with open(path,'r',encoding='utf-8') as html:
        soup = BeautifulSoup(html, "lxml") 
        print(soup.select('class.img.width.200'))
        
#     return [imgs, headers, linkslen, lists]

parse(path)

[]


In [ ]:
from bs4 import BeautifulSoup
import unittest

def parse(path):
    with open(path,'r',encoding='utf-8') as html:
        soup = BeautifulSoup(html, "lxml")    
    # Поместите ваш код здесь.
    # ВАЖНО!!!
    # При открытии файла, добавьте в функцию open необязательный параметр
    # encoding='utf-8', его отсутствие в коде будет вызвать падение вашего
    # решения на грейдере с ошибкой UnicodeDecodeError
    return [imgs, headers, linkslen, lists]


class TestParse(unittest.TestCase):
    def test_parse(self):
        test_cases = (
            ('wiki/Stone_Age', [13, 10, 12, 40]),
            ('wiki/Brain', [19, 5, 25, 11]),
            ('wiki/Artificial_intelligence', [8, 19, 13, 198]),
            ('wiki/Python_(programming_language)', [2, 5, 17, 41]),
            ('wiki/Spectrogram', [1, 2, 4, 7]),)

        for path, expected in test_cases:
            with self.subTest(path=path, expected=expected):
                self.assertEqual(parse(path), expected)


if __name__ == '__main__':
    unittest.main()